In [6]:
import tensorflow as tf
import numpy as np
import sklearn
import random,os,io
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Dense

In [90]:
from tensorflow.keras.callbacks import Callback, EarlyStopping, LearningRateScheduler, ModelCheckpoint,CSVLogger
from tensorboard.plugins.hparams import api as hp

In [4]:
import pandas as pd
dep_train=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\XTr.dat",sep='\s+',names=[str(i) for i in range(0,16)])
indep_train=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\yTr.dat",sep='\s+',names=['target'])
dep_val=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\XV.dat",sep='\s+',names=[str(i) for i in range(0,16)])
indep_val=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\yV.dat",sep='\s+',names=["target"])
dep_test=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\XT.dat",sep='\s+',names=[str(i) for i in range(0,16)])
indep_test=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\yT.dat",sep='\s+',names=["target"])

dep_train.shape,indep_train.shape,dep_val.shape,indep_val.shape,dep_test.shape,indep_test.shape

((42039, 16), (42039, 1), (10935, 16), (10935, 1), (10934, 16), (10934, 1))

In [5]:
# Hyper-parameters
batch_size = 50
num_hidden_1 = 12
num_hidden_2 = 12
learning_rate = 0.0001
training_epochs = 100
dropout_keep_prob = 0.3 # set to no dropout by default

# variable to control the resolution at which the training results are stored
display_step = 1

In [81]:
reg = tf.keras.Sequential()
reg.add(Dense(units = 6, activation = 'relu', input_dim = 16))

In [73]:
reg.add(Dense(units = 6, activation = 'relu'))

In [74]:
reg.add(Dense(units = 6, activation = 'relu'))

In [75]:
reg.add(Dense(units = 1, activation = 'linear'))

In [82]:
reg.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 6)                 102       
                                                                 
Total params: 102
Trainable params: 102
Non-trainable params: 0
_________________________________________________________________


In [77]:
optimizer = tf.keras.optimizers.Adam()
reg.compile(optimizer=optimizer,  
            loss='mean_absolute_error')

In [79]:
reg.fit(dep_train, indep_train, batch_size = 10, epochs = 10)

Epoch 1/10
4204/4204 [==============================] - 6s 1ms/step - loss: 0.0043
Epoch 2/10
4204/4204 [==============================] - 8s 2ms/step - loss: 9.9983e-04
Epoch 3/10
4204/4204 [==============================] - 12s 3ms/step - loss: 8.0338e-04
Epoch 4/10
4204/4204 [==============================] - 13s 3ms/step - loss: 6.5178e-04
Epoch 5/10
4204/4204 [==============================] - 13s 3ms/step - loss: 5.4190e-04
Epoch 6/10
4204/4204 [==============================] - 13s 3ms/step - loss: 5.0964e-04
Epoch 7/10
4204/4204 [==============================] - 12s 3ms/step - loss: 4.8497e-04
Epoch 8/10
4204/4204 [==============================] - 10s 2ms/step - loss: 4.5276e-04
Epoch 9/10
4204/4204 [==============================] - 11s 3ms/step - loss: 4.5096e-04
Epoch 10/10
4204/4204 [==============================] - 12s 3ms/step - loss: 4.2349e-04


In [20]:
reg.save("demo")

INFO:tensorflow:Assets written to: demo\assets


In [21]:
reg_reload=tf.keras.models.load_model("demo")
reg_reload.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 102       
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dense_2 (Dense)             (None, 6)                 42        
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


In [22]:
reg_reload.predict(dep_test)

342/342 [==============================] - 2s 4ms/step


array([[0.01661042],
       [0.01399928],
       [0.01650758],
       ...,
       [0.01186464],
       [0.01621098],
       [0.00988809]], dtype=float32)

In [116]:
class regclass(Model):
    def __init__(self):
        super(regclass,self).__init__()
        self.W1 = self.add_weight(shape = (16,6), initializer = 'random_normal', trainable = True)
        self.b1 = self.add_weight(shape = (6,), initializer = 'random_normal', trainable = True)
        self.dp1 = tf.keras.layers.Dropout(0.2)
        self.W2 = self.add_weight(shape = (6,6), initializer = 'random_normal', trainable = True)
        self.b2 = self.add_weight(shape = (6,), initializer = 'random_normal', trainable = True)
        self.dp2 = tf.keras.layers.Dropout(0.2)
        self.W3 = self.add_weight(shape = (6,1), initializer = 'random_normal', trainable = True)
        self.b3 = self.add_weight(shape = (1,), initializer = 'random_normal', trainable = True)
    

    def call(self,x):
        out = x

        out = tf.nn.relu(tf.matmul(out, self.W1) + self.b1)
        out = self.dp1(out)
        out = tf.nn.relu(tf.matmul(out, self.W2) + self.b2)
        out = self.dp2(out)
        out = tf.matmul(out, self.W3) + self.b3

        return out

In [62]:
class cus(Model):
    def __init__(self) -> None:
        super(cus,self).__init__()
        self.dense1= Dense(units = 6, activation = 'relu', input_dim = 16)
        self.dense2= Dense(units = 6, activation = 'relu')
        self.dense3= Dense(units = 6, activation = 'relu')
        self.dense4= Dense(units = 1, activation = 'linear')

    def call(self,x):
        out = self.dense1(x)
        out = self.dense2(out)
        out = self.dense3(out)
        out = self.dense4(out)

        return out

In [117]:
reg_cus= regclass()

In [118]:
reg_cus.compile(optimizer=optimizer,  
            loss='mean_absolute_error')

In [119]:
reg_cus.fit(dep_train, indep_train, epochs = 2)

Epoch 1/2
1314/1314 [==============================] - 14s 9ms/step - loss: 0.0012
Epoch 2/2
1314/1314 [==============================] - 5s 4ms/step - loss: 0.0010


In [115]:
reg_cus.summary()

Model: "regclass_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_1 (Dropout)         multiple                  0         
                                                                 
Total params: 151
Trainable params: 151
Non-trainable params: 0
_________________________________________________________________


In [42]:
class SobolevNetwork(Model):
    def __init__(self, input_dim, num_hidden,init = None):
        super(SobolevNetwork, self).__init__()
        self.input_dim = input_dim 
        self.num_hidden = num_hidden
        self.W1 = tf.Variable(tf.random.normal([self.input_dim, self.num_hidden],stddev=0.1))
        self.b1 = tf.Variable(tf.ones([self.num_hidden]))
        self.W2 = tf.Variable(tf.random.normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b2 = tf.Variable(tf.ones([self.num_hidden]))
        self.W3 = tf.Variable(tf.random.normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b3 = tf.Variable(tf.ones([self.num_hidden]))
        self.W4 = tf.Variable(tf.random.normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b4 = tf.Variable(tf.ones([self.num_hidden]))
        self.W5 = tf.Variable(tf.random.normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b5 = tf.Variable(tf.ones([self.num_hidden]))
        self.W6 = tf.Variable(tf.random.normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b6 = tf.Variable(tf.ones([self.num_hidden]))        
        self.W7 = tf.Variable(tf.random.normal([self.num_hidden, 1],stddev=0.1))
        self.b7 = tf.Variable(tf.ones([1]))
        self.w = [(self.W1, self.b1), (self.W2, self.b2), (self.W3, self.b3),(self.W4, self.b4), (self.W5, self.b5), (self.W6, self.b6),(self.W7, self.b7)]
        
    def call(self, X):
        #Input layer
        out = X
        #Hidden layers
        W,b = self.w[0]
        out = tf.nn.tanh(tf.matmul(out, W) + b)
        W,b = self.w[1]
        out = tf.nn.tanh(tf.matmul(out, W) + b)
        W,b = self.w[2]
        out = tf.nn.sigmoid(tf.matmul(out, W) + b)
        W,b = self.w[3]
        out = tf.nn.sigmoid(tf.matmul(out, W) + b)
        W,b = self.w[4]
        out = tf.nn.leaky_relu(tf.matmul(out, W) + b)
        W,b = self.w[5]
        out = tf.nn.relu(tf.matmul(out, W) + b)
        #Output layer
        W,b = self.w[-1]
        out = tf.matmul(out, W) + b
        return out


In [102]:
class LossCallback(Callback):
    def on_epoch_end(self, epoch, logs):
        print("\n For Epoch Number {} the model has loss[mean_absolute_error] of {}".format(epoch+1, logs["loss"]))

    def on_batch_end(self, batch, logs):
        print("\n For Batch Number {} the model has loss[mean_absolute_error] of {}".format(batch+1, logs))

In [92]:
csv_callback = CSVLogger( 'logs.csv',separator=',',append=True)

In [100]:
es_callback = EarlyStopping( monitor='val_loss',min_delta=0,patience=2, verbose=1, mode="auto", baseline=None, restore_best_weights=False)

In [96]:
def schedular(epoch, lr):
    if epoch <= 3:
        learning_rate = lr
    else:
        learning_rate = lr * tf.math.exp(-0.1)
        learning_rate = learning_rate.numpy()
    

    # with train_writer.as_default():
    #     tf.summary.scalar('LearningRateScheduler', data= learning_rate, step= epoch)
    
    return learning_rate

schedular_callback = LearningRateScheduler(schedular, verbose=1)

In [43]:
custom_snn= SobolevNetwork(input_dim=16,num_hidden=num_hidden_1)

In [44]:
custom_snn.compile(optimizer=optimizer,  
            loss='mean_absolute_error')

In [103]:
custom_snn.fit(dep_train, indep_train, validation_data=(dep_val,indep_val),batch_size = 1000, epochs = 8, callbacks=[schedular_callback, es_callback,LossCallback(), csv_callback])


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/8

 For Batch Number 1 the model has loss[mean_absolute_error] of {'loss': 0.0011676228605210781}
 1/43 [..............................] - ETA: 0s - loss: 0.0012
 For Batch Number 2 the model has loss[mean_absolute_error] of {'loss': 0.0011432855390012264}

 For Batch Number 3 the model has loss[mean_absolute_error] of {'loss': 0.0010535717010498047}

 For Batch Number 4 the model has loss[mean_absolute_error] of {'loss': 0.0011435628402978182}

 For Batch Number 5 the model has loss[mean_absolute_error] of {'loss': 0.0012263270327821374}
 5/43 [==>...........................] - ETA: 0s - loss: 0.0012
 For Batch Number 6 the model has loss[mean_absolute_error] of {'loss': 0.0011148732155561447}

 For Batch Number 7 the model has loss[mean_absolute_error] of {'loss': 0.001069816411472857}

 For Batch Number 8 the model has loss[mean_absolute_error] of {'loss': 0.0010683939326554537}

 For Batch Numbe

In [46]:
custom_snn.summary()

Model: "sobolev_network_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 997
Trainable params: 997
Non-trainable params: 0
_________________________________________________________________


In [109]:
tf.keras.utils.plot_model(custom_snn, show_shapes=True, rankdir="LR")

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [110]:
pip install graphviz

In [107]:
custom_snn.evaluate(dep_test, indep_test)

342/342 [==============================] - 2s 6ms/step - loss: 0.0013


0.0013376888819038868